# **Procesado básico de la señal: Filtrado**
## Procesamiento de Imagen y Señal
### **ESCRIBE TU NOMBRE AQUI**
### Curso 2024-2025

In [ ]:
import math
import cmath

import os
import sys

import random

##

import warnings
warnings.filterwarnings('ignore')

##

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.image as im

from scipy import signal, fftpack, ndimage
from scipy.fft import fft, ifft

import scipy.signal as sp_s
import scipy.io as sp_io

from scipy.io import wavfile

from scipy.ndimage import convolve

### Audio
from IPython.display import Image, HTML, display
import IPython.display as ipd
import librosa
import librosa.display

from pydub import AudioSegment
from pydub.effects import normalize
from pydub.playback import play

import soundfile as sf

#### Important tips
# To hear the signal
# ipd.Audio(x, rate=fs)
####

### Image
from PIL import Image
from skimage import io, transform, img_as_ubyte, img_as_float, data, exposure
import cv2

In [ ]:
sys.path.append(os.getcwd())

# Funciones auxiliares
path = "code/"

# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(path))

from f_audio_player_list import *
from f_plot import *
from f_get_random import *

---

# **Introducción**

El proceso por el cual una determinada señal se modifica (se atenúa o se elimina una  parte  de  su  espectro  de  frecuencias)  se  denomina  **filtrado**,  y  al  sistema  que  realiza  dicha acción se le llama **filtro**. 

El rango de frecuencias atenuadas o filtradas es lo que se conoce como **banda de rechazo**, mientras que las restantes frecuencias forman lo que se llama **banda de paso**.

---

# **Filtros paso-bajo, paso-alto, paso-banda**

En función de como sea la banda de paso, se pueden definir los siguientes tipos de filtro:

* Filtro paso-bajo, cuando se atenúan todas las frecuencias por encima de una dada. 
* Filtro paso-alto, cuando se atenúan todas las frecuencias por debajo de una dada.
* Filtro paso-banda, cuando se atenúan todas las frecuencias que no estén entre dos valores dados.
* Filtro rechazo-banda o suprime-banda (*band-stop*), cuando se atenúan todas las frecuencias que estén entre dos valores dados.

---

# **Filtros FIR e IIR**

## **Filtro FIR**

Un filtro de tipo FIR (*Finite Impulse Response*) es una operación lineal sobre una señal discreta que produce respuestas de impulso finitas. 

La base de los filtros FIR consiste en conectar la entrada del filtro a una serie de retardos. El primero con sólo una muestra de retraso, el segundo con dos, el tercero con tres y así sucesivamente. En seguida se amplifica o atenúa cada retraso con un factor de ganancia específico y finalmente se suman las salidas. Al organizar estas ganancias una detrás de otra obtenemos la respuesta impulso del filtro.

Si se toma la transformada Z se puede representar la respuesta en frecuencia del filtro, la cual depende completamente de los "*taps*" o coeficientes del filtro o factores de ganancias o valores de la respuesta impulso.

$$\text{Finite Impulse Response (FIR) Filter: }y[n]=\sum^{M}_{k=0}b[k]x[n-k]$$

donde los $(M+1)$ coeficientes son $b[n]$ con $n=0, ..., M$.

<hr style="border: 1px solid green" />

### **Ejemplo**

Vamos a aplicar un filtro FIR "*band-stop*" a una señal de audio `audio_pipeline/aircrew.wav`.

In [ ]:
ipd.Audio('audio_pipeline/aircrew.wav')

In [ ]:
# scipy.signal == sp_s

x, fs = sf.read('audio_pipeline/aircrew.wav')

# fs_nyquist = ...

# Ancho de la region de transicion (50Hz / frecuencia Nyquist)
# width = ...

# ondulacion
ripple_db = 60 # atenuacion en dB

# Filtro FIR (orden y parametro Kaiser)
# Buscamos los parametros de la ventana de filtro para el metodo de ventana de Kaiser
# https://es.wikipedia.org/wiki/Ventana_de_Kaiser

# N, beta = sp_s.kaiserord(ripple_db, width)

# Band-stop en Hz
bandas = [1200, 1500]

# Filtro con una ventana Kaiser para crear un filtro FIR paso-bajo
# parametros: numero-taps y cutoff

# Respuesta del filtro donde "w" es la matriz de frecuencias y "h" es la matriz 
# compleja correspondiente de respuestas de frecuencia

# Convert w to Hz

# Dibujamos el filtro con la funcion: plot_FIR_filter_spec()

# Aplicamos el filtro a la senal de audio y lo oimos

Como vemos, la grabación contiene mucho ruido de fondo a altas frecuencias. Por ello, vamos a diseñar un filtro paso-bajo para filtrar la señal.

In [ ]:
# scipy.signal == sp_s

# Banda stop en Hz
bandas = [2100]  

# Filtro con una ventana Kaiser para crear un filtro FIR paso-bajo
# parametros: numero-taps y cutoff

# Respuesta del filtro donde "w" es la matriz de frecuencias y "h" es la matriz 
# compleja correspondiente de respuestas de frecuencia

# Convert w to Hz

# Dibujamos el filtro con la funcion: plot_FIR_filter_spec()

# Aplicamos el filtro a la senal de audio y lo oimos

Aplicando Fourier podemos realizar un filtrado similar.

In [ ]:
T_s = 1/fs
N_s = len(x)
t = np.arange(0, N_s*T_s, T_s)

X = fft(x)

cutoff = 2200
# band-stop [1200, 1500]
band_i = 1200
band_f = 1500
n   = round(cutoff/fs*N_s)     # calculate the frequency index
n_i = round(band_i/fs*N_s)
n_f = round(band_f/fs*N_s)

X[n:-n] = 0       # low-pass
X[n_i:n_f] = 0    # band-stop (1)
X[-n_f:-n_i] = 0  # band-stop (2)

y = np.real(ifft(X))
ipd.Audio(y, rate=fs)

---

## **Filtro IIR**

Un filtro de respuesta de impulso infinito (IIR, *Infinite Impulse Response*) tiene respuestas impulso de longitud infinita y la respuesta depende no solo de los valores de entrada actuales y anteriores, sino también de los valores de salida anteriores. Puede representarse mediante ecuaciones de diferencias (sumas y restas de entradas y salidas con retrasos), una función de transferencia o un diagrama de bloques. 

Un filtro IIR, además de utilizar retrasos para los valores a la entrada del filtro, toma también los valores de la salida, les aplica una nueva cadena de retrasos y retroalimenta esta señal a la entrada del filtro.

Los más comunes son los filtros *Butterworth*, *Chebyshev*, elípticos y de *Bessel*. Estos filtros tienen diferentes respuestas de frecuencia, ondulación de banda de paso y banda de parada, ancho de banda de transición y distorsión de fase. Dependiendo de su aplicación, puede elegir el tipo de filtro IIR que mejor se adapte a sus necesidades y especificaciones.  

Para diseñar un filtro IIR, debe especificar el orden del filtro, la frecuencia de corte y la atenuación de la banda de paso y de la banda de parada. Para encontrar los coeficientes del filtro se pueden utilizar diferentes técnicas como la transformada bilineal, método invariante de momento o el método de colocación de polo cero.

Una de las principales ventajas de los filtros IIR es que pueden lograr el mismo rendimiento de filtrado que los filtros FIR con orden inferior y menor complejidad computacional. Otra ventaja de los filtros IIR es que pueden aproximarse a los filtros analógicos, lo que los hace útiles para la conversión de analógico a digital y la reconstrucción de señales.

$$\text{Infinite Impulse Response (IIR) Filter: }y[n]=\sum^{M}_{k=0}b[k]x[n-k]-\sum^{N}_{k=1}a[k]y[n-k]$$

<hr style="border: 1px solid pink" />

En `Python` podemos utilizar la función `scipy.signal.filtfilt(b, a, x)` para generar filtros FIR e IIR donde $$b = \big[b[0],\: b[1], \cdots, b[1-M]\big]$$ y $$a=\big[1,\: a[1], a[2], \cdots, a[N-1]\big]$$.

Los filtros FIR pueden implementarse con esta función donde $a=[1]$

<hr style="border: 1px solid pink" />

### **Ejemplo**

In [ ]:
# scipy.signal == sp_s

fs = 44100
# band-stop
bandas = [1200, 1500]

N = 5

# fs_nyquist = ...

# Filtros band-stop (IIR) mediante la funcion butter() del paquete scipy.sinal

# Filtros paso-bajo (frecuencia de corte = 2000 Hz) y N = 15

# Oir la senal filtrada

<hr style="border: 1px solid green" />

2. Filtra el audio `audio_pipeline/bass_oboe_raphael.wav` de forma que el filtro "deje pasar" solo las frecuencias entre $1000$ Hz y $3000$ Hz. Visualiza y oye la señal filtrada. Visualiza también los espectros de la señal original y de la filtrada.
    La frecuencia $1000$ Hz, ¿ a qué "bin"" de la FFT corresponde ? ¿ Y la de $3000$ Hz ?
    
    Nota: El "bin" ($n$) al que pertenece una frecuencia ($f$) dada una frecuencia de muestreo ($fs$) y el tamaño de la ventana del análisis de Fourier ($N$) viene dado por la siguiente ecuación:

    $$n = \frac{f * N}{fs}$$

    * Respuesta: **Escribe aquí tu respuesta**

In [ ]:
# Filtra la señal, visualizala, oyela y visualiza los espectros de la señal original y de la señal filtrada.

In [ ]:
# Leer el fichero
file = "audio_pipeline/bass_oboe_raphael.wav"
x, fs = librosa.load(file, sr=sf.info(file).samplerate)

# Aplicar un filtro paso-banda de 1000-3000 Hz a  la senal original

# Dibujar la senal, el espectro y el espectrograma con la funcion plotN_wss() (ver fichero code/f_plot.y)

# Oir el audio original y el audio filtrado. Puedes utilizar la funcion audio_player_list() (ver fichero code/audio_player_list())

<hr style="border: 1px solid green" />

3. **Filtros espaciales**: 
    + Selecciona aleatoriamente una imagen del directorio `images_RGB`.
    + Aplícale un filtro de convolución gaussiano, un filtro de la mediana y un filtro del punto medio.

In [ ]:
# Realiza la tarea

In [ ]:
# Una aleatoria
n_folder = "images_RGB"
name_image = get_random_image_path(n_folder)
print(name_image)

# Leemos la imagen
im = io.imread(name_image)
im_f = img_as_float(im)

# Aplicar el filtro de convolucion gaussiano (filtro blur), un fichero de mediana y un filtro del punto medio.
# Visualizar la imagen original y las filtradas con la funcion plotN_im() (ver fichero code/f_plot.y)

---

4. **Separación de cantos de pájaros mediante filtrado**

+ Oye el audio `audio_natura/XC403881_Turpial dorsidorado_Icterus chrysater.mp3`
+ Separa dos zonas de frecuencias: $1700-2600$ Hz y $5100-6100$ Hz mediante operaciones de filtrado.

In [ ]:
# Una aleatoria
#n_folder = "audio_natura"
#name_audio = get_random_audio_path(n_folder)
#print(name_audio)

name_audio = "audio_natura/XC403881_Turpial dorsidorado_Icterus chrysater.mp3"

# Leemos el audio
x, fs = librosa.load(name_audio)
ipd.Audio(x, rate=fs)

In [ ]:
# Separa dos zonas de frecuencias: $1700-2600$ Hz y $5100-6100$ Hz mediante operaciones de filtrado
# Visualiza los espectros y espectrogramas del audio general y de los audios filtrados.
# Oir el audio original y el audio filtrado. Puedes utilizar la funcion audio_player_list() (ver fichero code/audio_player_list())

5. **Eliminación del ruido mediante filtrado**

+ Elimina el ruido de la línea eléctrica y sus armónicos.
+ La señal se encuentra en el fichero `data/lineNoiseData.mat` y puede leerse mediante la función del paquete `scipy.io` [**`loadmat()`**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html). Las frecuencias de ruido se encuentran a $50$ Hz, $150$ Hz y $250$ Hz como puede verse en el espectro de la señal.

In [ ]:
# Carga los datos
linedata = sp_io.loadmat('data/lineNoiseData.mat')
data  = np.squeeze(linedata['data'])
fs = linedata['srate'][0][0]

# Visualiza la senal y el espectro 
# Aplica un filtro para eliminar las frecuencia de ruido y sus armonicos (50 Hz, 150 Hz, 250 Hz)
# Visualiza la senal filtrada